In [115]:
import pandas as pd
import numpy as np
import os

In [116]:
data = pd.read_csv('Data - Sheet1.csv')

In [117]:
data.head(5)

,name,first_name,last_name,email,created_at,approval_status,has_joined_event,amount,amount_tax,amount_discount,currency,ticket_name,Job Title,What is your LinkedIn profile?
0,Venkatesh R,Venkatesh,R,*****@gmail.com,2025-01-12T07:14:38.302Z,approved,No,NaN,NaN,NaN,NaN,Standard,Freelance,https://linkedin.com/in/venkatesh-r-42845a282
1,Mark Jawut,Mark,Jawut,*****@gmail.com,2025-01-15T10:05:14.049Z,approved,No,$0.00,$0.00,$0.00,usd,Standard,Student,MarkJawut
2,Avinash Kumar,Avinash,Kumar,*****@sirmvit.edu,2025-01-13T17:11:21.147Z,approved,No,NaN,NaN,NaN,NaN,Standard,Student,linkedin.com/in/Avinash.kumar5167
3,Wilson Juma,Wilson,Juma,*****@student.embuni.ac.ke,2025-01-15T13:32:42.805Z,approved,No,$0.00,$0.00,$0.00,usd,Standard,Digital Marketing Specialist,linkedin.com/in/wilsonjuma254
4,Suneel,Suneel,NaN,*****@gmail.com,2025-01-14T14:01:11.567Z,approved,Yes,NaN,NaN,NaN,NaN,Standard,Devops engineer,NaN


In [118]:
data = data.drop_duplicates()

In [119]:
data_info = data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 643 entries, 0 to 646
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   name                            643 non-null    object
 1   first_name                      526 non-null    object
 2   last_name                       413 non-null    object
 3   email                           643 non-null    object
 4   created_at                      643 non-null    object
 5   approval_status                 643 non-null    object
 6   has_joined_event                643 non-null    object
 7   amount                          382 non-null    object
 8   amount_tax                      382 non-null    object
 9   amount_discount                 382 non-null    object
 10  currency                        382 non-null    object
 11  ticket_name                     641 non-null    object
 12  Job Title                       641 non-null    object


In [120]:
# Normalize 'has_joined_event' values
data['has_joined_event'] = data['has_joined_event'].str.strip().str.lower().map({'yes': True, 'no': False})

In [121]:
# Flag missing or blank job titles
# df['missing_job_title'] = df['Job Title'].apply(lambda x: pd.isna(x) or str(x).strip() == '')

In [122]:
# Flag incomplete or missing LinkedIn profiles
def is_linkedin_incomplete(value):
    if pd.isna(value):
        return True
    value = str(value).strip().lower()
    return not (value.startswith("http://") or value.startswith("https://"))

data['missing_linkedin'] = data['What is your LinkedIn profile?'].apply(is_linkedin_incomplete)

In [123]:
def clean_linkedin(value):
    if pd.isna(value):
        return ""  # Replace NaN with empty string
    value = str(value).strip()
    if value.lower().startswith("http://") or value.lower().startswith("https://"):
        return value
    return ""  # Replace bad URLs with empty string

# Apply cleaning
data['What is your LinkedIn profile?'] = data['What is your LinkedIn profile?'].apply(clean_linkedin)

In [124]:
data['approval_status'].value_counts()

approval_status
approved    641
declined      2
Name: count, dtype: int64

In [125]:
data['ticket_name'].value_counts()

ticket_name
Standard    641
Name: count, dtype: int64

In [ ]:
data['Job Title'].value_counts()

Job Title
Student                      105
Data analyst                  13
Developer                     12
student                       12
Data scientist                11
                            ... 
Product Marketer               1
Junior Software Engineer       1
Machine Learning Engineer      1
None yet, an enthusiast        1
Principal product manager      1
Name: count, Length: 352, dtype: int64

In [127]:
# Convert created_at to datetime object
data['created_at'] = pd.to_datetime(data['created_at'], errors='coerce')
# # Format it as 'YYYY-MM-DD HH:MM AM/PM'
data['created_at'] = data['created_at'].dt.strftime('%B %d, %Y — %I:%M %p')

In [128]:
 # Fix inconsistent casing in Job Title: 'student' → 'Student'
data['Job Title'] = data['Job Title'].apply(lambda x: str(x).strip().capitalize() if pd.notna(x) else x)

In [129]:
# Fill missing job titles with most frequent value
most_common_job = data['Job Title'].mode()[0]
data['Job Title'] = data['Job Title'].fillna(most_common_job)

In [130]:
# Fill missing ticket_name with most frequent value
most_common_ticket = data['ticket_name'].mode()[0]
data['ticket_name'] = data['ticket_name'].fillna(most_common_ticket)

In [132]:
data['Job Title'].value_counts()

Job Title
Student             120
Data analyst         21
Data scientist       18
Developer            13
Consultant            9
                   ... 
Website designer      1
Ai/ml                 1
Program managar       1
College student       1
Office of ceo         1
Name: count, Length: 316, dtype: int64

In [133]:
 
# 1. Standardize casing and trim
data['name']        = data['name'].str.strip().str.title()
data['first_name']  = data['first_name'].str.strip().str.title()
data['last_name']   = data['last_name'].str.strip().str.title()

# 2. Parse full name into parts
def parse_name(full):
    parts = str(full).split()
    if len(parts) >= 2:
        return parts[0], parts[-1]
    elif len(parts) == 1:
        return parts[0], ''
    else:
        return '', ''

data[['parsed_first', 'parsed_last']] = data['name'].apply(lambda x: pd.Series(parse_name(x)))

# 3. Back-fill missing first/last names
data['first_name'] = data['first_name'].fillna(data['parsed_first'])
data['last_name']  = data['last_name'].fillna(data['parsed_last'])

 
# (Optional) drop helpers if you want
data = data.drop(columns=['parsed_first', 'parsed_last'])

In [135]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 643 entries, 0 to 646
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   name                            643 non-null    object
 1   first_name                      643 non-null    object
 2   last_name                       643 non-null    object
 3   email                           643 non-null    object
 4   created_at                      643 non-null    object
 5   approval_status                 643 non-null    object
 6   has_joined_event                643 non-null    bool  
 7   amount                          382 non-null    object
 8   amount_tax                      382 non-null    object
 9   amount_discount                 382 non-null    object
 10  currency                        382 non-null    object
 11  ticket_name                     643 non-null    object
 12  Job Title                       643 non-null    object


In [137]:
# Save cleaned output
data.to_csv("cleaned_output.csv", index=False)